In [1]:
# Jessica Gallo
# CSC767 Neural Networks & Deep Learning
# Project 2
# Part E

In [2]:
# This is the original code with RMSProp
# This was run with the original dataset under limited data files:
# 0.txt, 6.txt and 8.txt.

In [3]:
#! /usr/bin/env python
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from helper import data_helpers
from text_cnn.LW_text_cnn import TextCNN
from tensorflow.contrib import learn
import yaml
import math
import sys

C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [4]:
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)

In [5]:
# Parameters
# ==================================================

In [6]:
# Data loading params
tf.compat.v1.flags.DEFINE_float("dev_sample_percentage", 0.2, "Percentage of the training data to use for validation")

In [7]:
# Model Hyperparameters
tf.compat.v1.flags.DEFINE_boolean("enable_word_embeddings", False, "Enable/disable the word embedding (default: True)")
tf.compat.v1.flags.DEFINE_integer("embedding_dim", 200, "Dimensionality of character embedding (default: 128)")
tf.compat.v1.flags.DEFINE_string("filter_sizes", "2,3,5", "Comma-separated filter sizes (default: '2,3,5')")
tf.compat.v1.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 120)")
tf.compat.v1.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.compat.v1.flags.DEFINE_float("l2_reg_lambda", 0.0001, "L2 regularization lambda (default: 0.0)")

In [8]:
# Training parameters
tf.compat.v1.flags.DEFINE_integer("batch_size", 32, "Batch Size (default: 64)")
tf.compat.v1.flags.DEFINE_integer("num_epochs",10, "Number of training epochs (default: 10)")
tf.compat.v1.flags.DEFINE_integer("evaluate_every", 50, "Evaluate model on dev set after this many steps (default: 100)")
tf.compat.v1.flags.DEFINE_integer("checkpoint_every", 50, "Save model after this many steps (default: 100)")
tf.compat.v1.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

In [9]:
# Misc Parameters
tf.compat.v1.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.compat.v1.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.compat.v1.flags.DEFINE_float("decay_coefficient", 2.5, "Decay coefficient (default: 2.5)")

In [10]:
FLAGS = tf.compat.v1.flags.FLAGS

In [11]:
YML_PATH = os.path.join(ROOT_DIR, "train/helper/config.yml")
print("***********************YML_PATH", YML_PATH)

***********************YML_PATH C:\Users\User\Lightweighted-CNN-for-Document-Classification\train/helper/config.yml


In [12]:
with open(YML_PATH, 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

In [13]:
# ===================================================================================
# added this because of 'Unrecognizable flag error: Unknown command line flag f'
tf.compat.v1.flags.DEFINE_string('f','','')
# ===================================================================================

dataset_name = cfg["datasets"]["default"]
if FLAGS.enable_word_embeddings and cfg['word_embeddings']['default'] is not None:
    embedding_name = cfg['word_embeddings']['default']
    embedding_dimension = cfg['word_embeddings'][embedding_name]['dimension']
else:
    embedding_dimension = FLAGS.embedding_dim

In [14]:
# Data Preparation
# ==================================================

# Load data
print("Loading data...")
datasets = None
if dataset_name == "tobacco":
    data_path = os.path.join(ROOT_DIR, cfg["datasets"][dataset_name]['parent_dir'] +'/')
    print("*************data_path", data_path)    
    datasets = data_helpers.get_datasets_tobacco(data_path)

elif dataset_name == "localdata":
    datasets = data_helpers.get_datasets_localdata(container_path=cfg["datasets"][dataset_name]["container_path"],
                                                     categories=cfg["datasets"][dataset_name]["categories"],
                                                     shuffle=cfg["datasets"][dataset_name]["shuffle"],
                                                     random_state=cfg["datasets"][dataset_name]["random_state"])

x_text, y = data_helpers.load_data_labels(datasets)

Loading data...
*************data_path C:\Users\User\Lightweighted-CNN-for-Document-Classification\data/tobacco-data/
['0.txt', '6.txt', '8.txt']


In [15]:
# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
# TODO: This is very crude, should use cross-validation
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Vocabulary Size: 16099
Train/Dev split: 9549/2387


In [16]:
# Training
# ==================================================

In [18]:
# ===========================================================================
# Added to fix error
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# ===========================================================================

with tf.Graph().as_default():
    session_conf = tf.compat.v1.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)

    sess = tf.compat.v1.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=embedding_dimension,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)
       
        global_step = tf.compat.v1.Variable(0, name="global_step", trainable=False)
        # ===========================================================================================
        # Optimizers
        early_train_op= tf.compat.v1.train.AdamOptimizer(cnn.learning_rate)
        # late_train_op = tf.compat.v1.train.MomentumOptimizer(cnn.learning_rate,0.9,use_nesterov=True) 
        late_train_op = tf.compat.v1.train.RMSPropOptimizer(cnn.learning_rate, decay=0.9, momentum=0.0) 
        # ===========================================================================================
        
        grads_and_vars_1 = early_train_op.compute_gradients(cnn.loss)
        train_op_1 = early_train_op.apply_gradients(grads_and_vars_1, global_step=global_step)
        
        grads_and_vars_2 = late_train_op.compute_gradients(cnn.loss)
        train_op_2 = late_train_op.apply_gradients(grads_and_vars_2, global_step=global_step)  

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.compat.v1.summary.scalar("loss", cnn.loss)
        acc_summary = tf.compat.v1.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.compat.v1.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.compat.v1.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.compat.v1.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.compat.v1.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.compat.v1.train.Saver(tf.compat.v1.trainable_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.compat.v1.global_variables_initializer())
        if FLAGS.enable_word_embeddings and cfg['word_embeddings']['default'] is not None:
            vocabulary = vocab_processor.vocabulary_
            initW = None
            if embedding_name == 'word2vec':
                # load embedding vectors from the word2vec
                print("Load word2vec file {}".format(cfg['word_embeddings']['word2vec']['path']))
                initW = data_helpers.load_embedding_vectors_word2vec(vocabulary,
                                                                     cfg['word_embeddings']['word2vec']['path'],
                                                                     cfg['word_embeddings']['word2vec']['binary'])
                print("word2vec file has been loaded")
            elif embedding_name == 'glove':
                # load embedding vectors from the glove
                print("Load glove file {}".format(cfg['word_embeddings']['glove']['path']))
                initW = data_helpers.load_embedding_vectors_glove(vocabulary,
                                                                  cfg['word_embeddings']['glove']['path'],
                                                                  embedding_dimension)
                print("glove file has been loaded\n")
            sess.run(cnn.W.assign(initW))

        def train_step(x_batch, y_batch, learning_rate):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob,                
              cnn.learning_rate: learning_rate              
            }
            
            #Conditional optimizer switch
            train_op = train_op_1 if tf.compat.v1.train.global_step(sess, global_step) <=(train_input_size/FLAGS.batch_size)*3 else train_op_2
                        
            _, step, summaries, loss, accuracy = sess.run(                
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],                
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            
            print("{}: step {}, loss {:g}, acc {:g}, learning_rate {:g}"
                  .format(time_str, step, loss, accuracy, learning_rate))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
                

        train_input_size=len(list(zip(x_train, y_train)))
        print('Trainning input set: x_train, y_train',train_input_size)
        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        
        # It uses dynamic learning rate with a high value at the beginning to speed up the training
        max_learning_rate = 0.004
        min_learning_rate = 0.0001        
        decay_speed = FLAGS.decay_coefficient*len(y_train)/FLAGS.batch_size
        
        # Training loop. For each batch...
        counter = 0
        print("*********Trainable PARAMETERS***********",np.sum([np.prod(v.get_shape().as_list()) for v in 
                                                                 tf.compat.v1.trainable_variables()]))
        for batch in batches:
            learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-counter/decay_speed)
            counter += 1
            x_batch, y_batch = zip(*batch)      
            
            if tf.compat.v1.train.global_step(sess, global_step) == ((train_input_size/FLAGS.batch_size)*3)+1: ##(2534*3)+1:
                min_learning_rate = 0.0035
                
            train_step(x_batch, y_batch, learning_rate)
            
            current_step = tf.compat.v1.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

(?, 41, 1, 1)
(?, 40, 1, 1)
(?, 42, 200, 1)
(?, 40, 1, 1)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Writing to C:\Users\User\Lightweighted-CNN-for-Document-Classification\train\runs\1607548227

Trainning input set: x_train, y_train 9549
*********Trainable PARAMETERS*********** 3224697
2020-12-09T16:10:31.615780: step 1, loss 1.13073, acc 0.3125, learning_rate 0.004
2020-12-09T16:10:32.212946: step 2, loss 1.04619, acc 0.4375, learning_rate 0.00399478
2020-12-09T16:10:32.784926: step 3, loss 1.03975, acc 0.34375, learning_rate 0.00398956
2020-12-09T16:10:33.377740: step 4, loss 1.14058, acc 0.375, learning_rate 0.00398435
2020-12-09T16:10:33.981153: step 5, loss 1.21297, acc 0.5, learning_rate 0.00397914
2020-12-09T16:10:34.568613: step 6, loss 1.09681, acc 0.4375, learning_rate 0.00397395
2020-12-09T16:10:35.168644: step 7, loss 1.08035, acc 0.34375, learning_rate 0.00396876
2020-12-09T16:10:35.758335: step 8, 

2020-12-09T16:11:26.080397: step 85, loss 1.05463, acc 0.4375, learning_rate 0.00358469
2020-12-09T16:11:26.681250: step 86, loss 0.845241, acc 0.59375, learning_rate 0.00358002
2020-12-09T16:11:27.263910: step 87, loss 0.795397, acc 0.6875, learning_rate 0.00357536
2020-12-09T16:11:27.860362: step 88, loss 0.817473, acc 0.59375, learning_rate 0.0035707
2020-12-09T16:11:28.442301: step 89, loss 0.85647, acc 0.5625, learning_rate 0.00356605
2020-12-09T16:11:28.999795: step 90, loss 0.664982, acc 0.78125, learning_rate 0.00356141
2020-12-09T16:11:29.596955: step 91, loss 0.756699, acc 0.625, learning_rate 0.00355677
2020-12-09T16:11:30.184001: step 92, loss 0.74835, acc 0.71875, learning_rate 0.00355214
2020-12-09T16:11:30.779378: step 93, loss 1.04438, acc 0.46875, learning_rate 0.00354752
2020-12-09T16:11:31.366152: step 94, loss 0.885096, acc 0.625, learning_rate 0.0035429
2020-12-09T16:11:31.950335: step 95, loss 0.688864, acc 0.6875, learning_rate 0.00353829
2020-12-09T16:11:32.5513

2020-12-09T16:12:25.420782: step 172, loss 0.802531, acc 0.6875, learning_rate 0.00320111
2020-12-09T16:12:26.018819: step 173, loss 0.566902, acc 0.78125, learning_rate 0.00319695
2020-12-09T16:12:26.601780: step 174, loss 0.704352, acc 0.6875, learning_rate 0.0031928
2020-12-09T16:12:27.190449: step 175, loss 0.763516, acc 0.75, learning_rate 0.00318866
2020-12-09T16:12:27.764460: step 176, loss 0.707481, acc 0.71875, learning_rate 0.00318452
2020-12-09T16:12:28.357401: step 177, loss 0.902625, acc 0.5625, learning_rate 0.00318039
2020-12-09T16:12:28.953124: step 178, loss 0.753949, acc 0.5625, learning_rate 0.00317626
2020-12-09T16:12:29.514588: step 179, loss 0.722815, acc 0.65625, learning_rate 0.00317214
2020-12-09T16:12:30.094613: step 180, loss 0.548255, acc 0.71875, learning_rate 0.00316803
2020-12-09T16:12:30.667342: step 181, loss 0.584278, acc 0.78125, learning_rate 0.00316392
2020-12-09T16:12:31.258755: step 182, loss 0.615858, acc 0.6875, learning_rate 0.00315981
2020-12-

2020-12-09T16:13:24.703966: step 259, loss 0.66735, acc 0.65625, learning_rate 0.00285975
2020-12-09T16:13:25.286967: step 260, loss 0.801868, acc 0.65625, learning_rate 0.00285605
2020-12-09T16:13:25.867451: step 261, loss 0.77187, acc 0.71875, learning_rate 0.00285236
2020-12-09T16:13:26.445936: step 262, loss 0.647984, acc 0.6875, learning_rate 0.00284867
2020-12-09T16:13:27.031204: step 263, loss 0.977967, acc 0.46875, learning_rate 0.00284499
2020-12-09T16:13:27.608051: step 264, loss 0.772406, acc 0.59375, learning_rate 0.00284131
2020-12-09T16:13:28.183522: step 265, loss 0.724135, acc 0.5625, learning_rate 0.00283764
2020-12-09T16:13:28.797542: step 266, loss 0.790715, acc 0.625, learning_rate 0.00283397
2020-12-09T16:13:29.382320: step 267, loss 0.7362, acc 0.6875, learning_rate 0.00283031
2020-12-09T16:13:29.970324: step 268, loss 0.481774, acc 0.75, learning_rate 0.00282665
2020-12-09T16:13:30.550087: step 269, loss 0.66335, acc 0.6875, learning_rate 0.002823
2020-12-09T16:1

2020-12-09T16:14:18.860457: step 345, loss 0.407239, acc 0.8125, learning_rate 0.00255926
2020-12-09T16:14:19.432958: step 346, loss 0.540997, acc 0.75, learning_rate 0.00255596
2020-12-09T16:14:20.013471: step 347, loss 0.36816, acc 0.9375, learning_rate 0.00255268
2020-12-09T16:14:20.584974: step 348, loss 0.381735, acc 0.78125, learning_rate 0.00254939
2020-12-09T16:14:21.178513: step 349, loss 0.455517, acc 0.8125, learning_rate 0.00254611
2020-12-09T16:14:21.754854: step 350, loss 0.521646, acc 0.71875, learning_rate 0.00254283

Evaluation:
2020-12-09T16:14:25.039957: step 350, loss 0.617177, acc 0.720151

Saved model checkpoint to C:\Users\User\Lightweighted-CNN-for-Document-Classification\train\runs\1607548227\checkpoints\model-350

2020-12-09T16:14:26.498318: step 351, loss 0.520172, acc 0.71875, learning_rate 0.00253956
2020-12-09T16:14:27.071422: step 352, loss 0.477961, acc 0.78125, learning_rate 0.00253629
2020-12-09T16:14:27.656452: step 353, loss 0.534409, acc 0.78125, le

...

2020-12-09T16:38:30.548910: step 2586, loss 0.0741015, acc 0.96875, learning_rate 0.000221956
2020-12-09T16:38:31.128910: step 2587, loss 0.0868055, acc 1, learning_rate 0.000221792
2020-12-09T16:38:31.690640: step 2588, loss 0.150261, acc 0.96875, learning_rate 0.000221629
2020-12-09T16:38:32.242215: step 2589, loss 0.116668, acc 0.9375, learning_rate 0.000221466
2020-12-09T16:38:32.806344: step 2590, loss 0.326354, acc 0.90625, learning_rate 0.000221303
2020-12-09T16:38:33.385787: step 2591, loss 0.237806, acc 0.90625, learning_rate 0.000221141
2020-12-09T16:38:33.938286: step 2592, loss 0.0942015, acc 0.96875, learning_rate 0.000220979
2020-12-09T16:38:34.480287: step 2593, loss 0.150904, acc 0.9375, learning_rate 0.000220817
2020-12-09T16:38:35.042315: step 2594, loss 0.358031, acc 0.875, learning_rate 0.000220655
2020-12-09T16:38:35.605419: step 2595, loss 0.182794, acc 0.9375, learning_rate 0.000220493
2020-12-09T16:38:36.177173: step 2596, loss 0.142983, acc 0.96875, learning_ra

2020-12-09T16:39:25.936311: step 2671, loss 0.0729228, acc 1, learning_rate 0.000208823
2020-12-09T16:39:26.480506: step 2672, loss 0.290698, acc 0.8125, learning_rate 0.000208677
2020-12-09T16:39:27.022002: step 2673, loss 0.298058, acc 0.875, learning_rate 0.000208531
2020-12-09T16:39:27.588973: step 2674, loss 0.244854, acc 0.9375, learning_rate 0.000208386
2020-12-09T16:39:28.149379: step 2675, loss 0.0722625, acc 1, learning_rate 0.000208241
2020-12-09T16:39:28.692814: step 2676, loss 0.0896786, acc 0.96875, learning_rate 0.000208096
2020-12-09T16:39:29.237459: step 2677, loss 0.156008, acc 0.9375, learning_rate 0.000207951
2020-12-09T16:39:29.789908: step 2678, loss 0.120143, acc 0.90625, learning_rate 0.000207806
2020-12-09T16:39:30.343240: step 2679, loss 0.182498, acc 0.90625, learning_rate 0.000207662
2020-12-09T16:39:30.907174: step 2680, loss 0.233804, acc 0.9375, learning_rate 0.000207518
2020-12-09T16:39:31.474169: step 2681, loss 0.175274, acc 0.90625, learning_rate 0.00

2020-12-09T16:40:20.496092: step 2756, loss 0.0326428, acc 1, learning_rate 0.000197104
2020-12-09T16:40:21.042590: step 2757, loss 0.247729, acc 0.875, learning_rate 0.000196974
2020-12-09T16:40:21.587127: step 2758, loss 0.206516, acc 0.84375, learning_rate 0.000196844
2020-12-09T16:40:22.128008: step 2759, loss 0.231474, acc 0.875, learning_rate 0.000196714
2020-12-09T16:40:22.692430: step 2760, loss 0.183869, acc 0.875, learning_rate 0.000196584
2020-12-09T16:40:23.251381: step 2761, loss 0.433062, acc 0.8125, learning_rate 0.000196455
2020-12-09T16:40:23.797417: step 2762, loss 0.188844, acc 0.9375, learning_rate 0.000196326
2020-12-09T16:40:24.364719: step 2763, loss 0.179587, acc 0.875, learning_rate 0.000196197
2020-12-09T16:40:24.922714: step 2764, loss 0.365251, acc 0.8125, learning_rate 0.000196068
2020-12-09T16:40:25.477682: step 2765, loss 0.0858666, acc 0.96875, learning_rate 0.000195939
2020-12-09T16:40:26.034218: step 2766, loss 0.194496, acc 0.96875, learning_rate 0.00

2020-12-09T16:41:13.084112: step 2843, loss 0.154897, acc 0.96875, learning_rate 0.000186415
2020-12-09T16:41:13.632371: step 2844, loss 0.163213, acc 0.96875, learning_rate 0.000186299
2020-12-09T16:41:14.202839: step 2845, loss 0.463589, acc 0.78125, learning_rate 0.000186184
2020-12-09T16:41:14.758762: step 2846, loss 0.355218, acc 0.90625, learning_rate 0.000186068
2020-12-09T16:41:15.363244: step 2847, loss 0.21327, acc 0.90625, learning_rate 0.000185953
2020-12-09T16:41:15.914220: step 2848, loss 0.447403, acc 0.90625, learning_rate 0.000185838
2020-12-09T16:41:16.479580: step 2849, loss 0.22713, acc 0.9375, learning_rate 0.000185723
2020-12-09T16:41:17.049184: step 2850, loss 0.171475, acc 0.90625, learning_rate 0.000185608

Evaluation:
2020-12-09T16:41:20.288319: step 2850, loss 1.01559, acc 0.712191

Saved model checkpoint to C:\Users\User\Lightweighted-CNN-for-Document-Classification\train\runs\1607548227\checkpoints\model-2850

2020-12-09T16:41:21.761320: step 2851, loss 0.0

2020-12-09T16:42:08.816466: step 2928, loss 0.113411, acc 0.9375, learning_rate 0.000177109
2020-12-09T16:42:09.380503: step 2929, loss 0.334181, acc 0.84375, learning_rate 0.000177006
2020-12-09T16:42:09.963925: step 2930, loss 0.327443, acc 0.875, learning_rate 0.000176903
2020-12-09T16:42:10.509306: step 2931, loss 0.316267, acc 0.90625, learning_rate 0.0001768
2020-12-09T16:42:11.050924: step 2932, loss 0.125715, acc 0.9375, learning_rate 0.000176697
2020-12-09T16:42:11.597641: step 2933, loss 0.125028, acc 0.96875, learning_rate 0.000176594
2020-12-09T16:42:12.159542: step 2934, loss 0.165859, acc 0.875, learning_rate 0.000176491
2020-12-09T16:42:12.711542: step 2935, loss 0.204309, acc 0.96875, learning_rate 0.000176389
2020-12-09T16:42:13.281343: step 2936, loss 0.189016, acc 0.84375, learning_rate 0.000176287
2020-12-09T16:42:13.859503: step 2937, loss 0.208817, acc 0.90625, learning_rate 0.000176184
2020-12-09T16:42:14.406998: step 2938, loss 0.0851074, acc 0.96875, learning_r

In [ ]:
# ===========================================================================
# Eval.py
# ===========================================================================

In [20]:
!python eval_SGD_small.py --eval_train --checkpoint_dir="./runs/1607548227/checkpoints/"

***********************YML_PATH C:\Users\User\Lightweighted-CNN-for-Document-Classification\train/helper/config.yml
dataset_name:  tobacco
Loading data !
*************data_path C:\Users\User\Lightweighted-CNN-for-Document-Classification\data/tobacco-data/
['0.txt', '6.txt', '8.txt']
Total number of test examples: 11936
Vocabulary Size: 16099
Train/Dev split: 9549/2387
size of x_dev, y_dev: 2387 2387

Evaluating...



C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (